<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#|-Preliminaries" data-toc-modified-id="|-Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>| Preliminaries</a></span></li><li><span><a href="#|-Cleaning" data-toc-modified-id="|-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>| Cleaning</a></span></li><li><span><a href="#|-Tokenization" data-toc-modified-id="|-Tokenization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>| Tokenization</a></span></li><li><span><a href="#|-POS-Tagging" data-toc-modified-id="|-POS-Tagging-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>| POS Tagging</a></span></li><li><span><a href="#|-Language-Detection" data-toc-modified-id="|-Language-Detection-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>| Language Detection</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Ratio-other-languages" data-toc-modified-id="Ratio-other-languages-5.0.1"><span class="toc-item-num">5.0.1&nbsp;&nbsp;</span>Ratio other languages</a></span></li></ul></li></ul></li><li><span><a href="#|-Linguistic-Processing" data-toc-modified-id="|-Linguistic-Processing-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>| Linguistic Processing</a></span><ul class="toc-item"><li><span><a href="#|-Lemmatization" data-toc-modified-id="|-Lemmatization-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>| Lemmatization</a></span></li></ul></li><li><span><a href="#|-Frequencies-for-categories" data-toc-modified-id="|-Frequencies-for-categories-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>| Frequencies for categories</a></span></li><li><span><a href="#|-Vectorisierung" data-toc-modified-id="|-Vectorisierung-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>| Vectorisierung</a></span><ul class="toc-item"><li><span><a href="#|-SpaCy" data-toc-modified-id="|-SpaCy-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>| SpaCy</a></span></li><li><span><a href="#|-Gensim" data-toc-modified-id="|-Gensim-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>| Gensim</a></span></li></ul></li></ul></div>

# | Preliminaries

In [ ]:
#! pip install spacy==3.0.0
#! python -m spacy download de_core_news_lg
#! pip3 install https://github.com/explosion/spacy-models/releases/download/de_core_news_lg-3.0.0/de_core_news_lg-3.0.0-py3-none-any.whl


In [ ]:
import pickle
import pandas as pd
import numpy as np
# Regex for Cleaning
import regex as re

# spaCy - Named Entity, Vecotorization
import spacy
nlp = spacy.load("de_core_news_lg")
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.tokens import Doc

# Tokenization
import nltk
from nltk.tokenize import RegexpTokenizer

# QOL
from tqdm import tqdm


# Gensim further testing Vectorization methods
import gensim.downloader as dl
import gensim
from gensim.models import KeyedVectors
from gensim.models.word2vec import Word2Vec

# Scikit learn for testing Clusteringmethods
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# Language Detection
# import langdetect

import warnings

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_pickle(r'/Users/anna/PycharmProjects/NLP/export.pickle')

# | Cleaning

In [ ]:
df.drop(['search_engine','edge','datetime'], axis=1, inplace=True)
df[:5]

In [ ]:
dictionary = {"Unnamed: 0" : "id"}
    
df = df.copy()
df.rename(columns=dictionary, inplace=True)

In [ ]:
# Unify terms
df['target_add'] = df['target_add'].apply(lambda x: re.sub("cv","lebenslauf", str(x)))
df['target_add'] = df['target_add'].apply(lambda x: re.sub("vs","versus", str(x)))
df['target_add'] = df['target_add'].apply(lambda x: re.sub("bvb","borussia dortmund", str(x)))

In [ ]:
# Strip whitespace
df['target_add'] = df['target_add'].str.split(',').str[-1].str.lstrip() # del whitespace
df['source_add'] = df['source_add'].str.split(',').str[-1].str.lstrip() # del whitespace

In [ ]:
df.head(5)

# | Tokenization

In [ ]:
# With RegexpTokenizer nltk module ->  take only tokens from words and numbers
from tqdm import tqdm
tqdm.pandas()
tokenizer = RegexpTokenizer(r'\w+')

df["tokens"] = df.progress_apply(lambda row: tokenizer.tokenize(str(row["target_add"].lower())), axis=1)

# | POS Tagging

In [ ]:
# Determine POS tags for tokens
import nltk
nltk.download('averaged_perceptron_tagger')
df["pos"] = df.progress_apply(lambda row: nltk.pos_tag(row["tokens"]), axis=1)


In [ ]:
df

# Entity Recognition

In [ ]:

##Enitity Recognition von Städten und Personen:
#https://spacy.io/usage/linguistic-features

#! pip install https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.0.0/de_core_news_sm-3.0.0-py3-none-any.whl
#! pip install https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.0.0/de_core_news_sm-3.0.0.tar.gz
import spacy
nlp = spacy.load('de_core_news_sm')
df['person'] = ''
df['location'] = ''
df['organization'] = ''
df['misc'] = ''
for row in df.itertuples(index=True, name='Pandas'):
    doc = nlp(str(row.target_add))
    for ent in doc.ents:
        if ent.label_ == 'LOC':
            df.at[row.Index, 'location'] = ent.text
        elif ent.label_ == 'MISC':
            df.at[row.Index, 'misc'] = ent.text
        elif ent.label_ == 'PER':
            df.at[row.Index, 'person'] = ent.text
        elif ent.label_ == 'ORG':
            df.at[row.Index, 'organization'] = ent.text

In [ ]:
df['location']
df.to_csv('out.csv', index=False, sep=';')

In [ ]:
pd.set_option('display.min_rows', 150)
pd.options.display.max_colwidth = 10000
print(df[['person', 'location', 'organization', 'misc']])

# | Linguistic Processing

## | Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('omw-1.4')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df["lemmata"] = df.apply(lambda row: [lemmatizer.lemmatize(word) for word in row["tokens"]], axis=1)

In [ ]:
# Lemmatization with word type from https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
# Lemmatize with POS Tag
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


df["lemmata_word_type"] = df.apply(lambda row: [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in row["tokens"]], axis=1)

# | x


In [ ]:
df['target_add']

In [ ]:
list1 = ['common terms','consulting gmbh','aktuelle teams','breaking news','eis cream','biontech impfung', 'aktueller verein', 'collapse full video', 'brandon williams', 'corona']

In [ ]:
list1

# Bi-grams 

In [ ]:
#!pip install https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.0.0/de_core_news_sm-3.0.0-py3-none-any.whl
#! pip install https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.0.0/de_core_news_sm-3.0.0.tar.gz
import re

import numpy as np

import pandas as pd

from pprint import pprint

import gensim

import gensim.corpora as corpora

from gensim.utils import simple_preprocess

from nltk.corpus import stopwords

from gensim.models import CoherenceModel

import spacy

#import pyLDAvis

#import pyLDAvis.gensim_models

import matplotlib.pyplot as plt

import nltk

import spacy
import nltk 
#nlp = spacy.load("de_core_news_sm-3.0.0")
nlp = spacy.load('de_core_news_sm',disable=['parser', 'ner'])

#importing the Stopwords to use them
from nltk.corpus import stopwords


german_stop_words = stopwords.words('german')
nltk.download('stopwords')
stop_words = stopwords.words('german')

stop_words.extend(['from', 'subject', 're', 'edu', 'use','for'])

#downloading the data
data = df['target_add']

processed_data = list(tokeniz(data))

#Building Bigram & Trigram Models

bigram = gensim.models.Phrases(processed_data, min_count=5, threshold=100)

trigram = gensim.models.Phrases(bigram[processed_data], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram_mod = gensim.models.phrases.Phraser(trigram)

#function to filter out stopwords

def remove_stopwords(texts):

    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#function to create bigrams

def create_bigrams(texts):

    return [bigram_mod[doc] for doc in texts]

#function to create trigrams

def create_trigrams(texts):

    [trigram_mod[bigram_mod[doc]] for doc in texts]

#function for lemmatization

def lemmatize(texts, allowed_postags=['NOUN', 'ADJ', 'VERB']):

    texts_op = []

    for sent in texts:

        doc = nlp(" ".join(sent))

        texts_op.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])

    return texts_op

#removing stopwords, creating bigrams and lemmatizing the text

data_wo_stopwords = remove_stopwords(processed_data)

data_bigrams = create_bigrams(data_wo_stopwords)

data_lemmatized = lemmatize(data_bigrams, allowed_postags=[ 'NOUN', 'ADJ', 'VERB'])

#printing the lemmatized data

print(data_lemmatized[:3])

#creating a dictionary

gensim_dictionary = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized

#building a corpus for the topic model

gensim_corpus = [gensim_dictionary.doc2bow(text) for text in texts]

#printing the corpus we created above.

print(gensim_corpus[:3]) 

#we can print the words with their frequencies.

[[(gensim_dictionary[id], freq) for id, freq in cp] for cp in gensim_corpus[:4]] 

#creating the LSI model 

lsi_model = gensim.models.lsimodel.LsiModel(

   corpus=gensim_corpus, id2word=gensim_dictionary, num_topics=20,chunksize=100

)

#viewing topics

pprint(lsi_model.print_topics())

# | Vectorisierung

## | SpaCy

In [ ]:
# https://www.shanelynn.ie/word-embeddings-in-python-with-spacy-and-gensim/
# https://stackoverflow.com/questions/62624284/how-to-save-word-vectors-in-spacy
# Testing Vectorization with a subset
df_2 = df.copy()[:5000]
token_list = df_2.tokens
# Creating Single Terms
token_list = [a for b in token_list for a in b]
token_list = token_list[:5000]

In [ ]:
# Creating Vector array for our subset
doc = list(nlp.pipe(token_list, disable=['parser', 'tagger', 'ner']))
vectors = [term.vector for term in doc]

In [ ]:
vectors

# Analyse

F: Enthalten von verschiedenen Suchmaschinen vorgeschlagene Search Query Suggestions einen Bias bei Personensuchen, z. B. im Hinblick auf Geschlecht, Berufsgruppe und Herkunft? Eine Untersuchung mithilfe von Recursive Algorithm Interrogation (RAI) und Alphabeterweiterung.

# Einfache lineare Regression.

In [ ]:
#Anhand der Grafik kann deskriptiv bestimmt werden wie der Verlauf ist (linear, quadratisch, kubisch, loess )
import numpy
import matplotlib.pyplot as plt
from ipywidgets import interactive
def f(n,t):
    x = [1,2,3,4,5,6,7,8] #hier kommen die UV rein (Anzahl der jeweiligen Gruppe (Sporter,Musiker, weitere Berufungen)
    #Die Gruppe beeinflusst die Begriffe
    y = [100,90,80,60,60,55,60,50] #hier kommen die A-Variablen rein (Anzahl der Begriffe eines Themengebiets (z.b. Familie))
# z = ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'C']
    grün = plt.scatter(x, y, c='green')
    
    x = [5,6,7,4,5,6,7,8] #hier kommen die UA-Variable rein (Anzahl der jeweiligen Gruppe)
    y = [99,90,80,60,60,55,60,50] 
    rot = plt.scatter(x, y, c='red')

    curve_model = numpy.poly1d(numpy.polyfit(x, y, n))

    curve_fitting_line = numpy.linspace(1, t, 100)
    plt.plot(curve_fitting_line, curve_model(curve_fitting_line))
    plt.legend((grün, rot), ('Sportler', 'Musiker'), scatterpoints=1,
           loc='lower left',
           ncol=3,
           fontsize=9)
    plt.title('Thema: Begriffe zur Familie von weiblicher Personen in Zusammenhang zu den Berufen')
    plt.ylabel("Frequencies (Familie)")
    plt.xlabel("Anzahl Berufe (gruppiert)")
    plt.show()


interactive_plot= interactive(f,n=(1,20),t = (1,30))
interactive_plot


In [ ]:
import pandas as pd
#Quelle: https://statologie.de/lineare-regression-python/
# Daten erstellen
df3 = pd.DataFrame({'Anzahl der Gruppen (Sportler, Musiker usw.)': [1, 2, 2, 4, 2, 1, 5, 4, 2, 4, 4, 3, 6, 5, 3, 4, 6, 2, 1, 2],
                   'Weiblich': [1, 3, 3, 5, 2, 2, 1, 1, 0, 3, 4, 3, 2, 4, 4, 4, 5, 1, 0, 1],
                   'Anzahl der Begriffe (Familie)': [76, 78, 85, 88, 72, 69, 94, 94, 88, 92, 90, 75, 96, 90, 82, 85, 99, 83, 62, 76]})

# Daten anzeigen 
df3

In [ ]:
#python -m pip install statsmodels 
import statsmodels.api as sm

# Antwortvariable definieren
y = df3['Anzahl der Begriffe (Familie)']

# Prädiktorvariablen definieren
x = df3[['Weiblich','Anzahl der Gruppen (Sportler, Musiker usw.)']]

# Konstante zu Prädiktorvariablen hinzufügen
x = sm.add_constant(x)

# lineares Regressionsmodell anpassen
model = sm.OLS(y, x).fit()

# Modellzusammenfassung anzeigen
print(model.summary())

#R-squared:
#F-statistic prüft, ob das Modell insgesamt einen Erklärungsbeitrag leistet.
#t: t-wert: 
#p: p-wert:
#Die Signifikanz (p-Wert) sollte einen möglichst kleinen Wert (<0,05) haben. Wenn dem so ist, leistet das Regressionsmodell einen Erklärungsbeitrag.

Beispiel Hypothese: Es gibt einen Zusammenhang zwischen des Themas Familie und weiblichen Sportlern oder Musikern.

0,0 < 0,1	kein Zusammenhang
0,1 < 0,3	geringer Zusammenhang
0,3 < 0,5	mittlerer Zusammenhang
0,5 < 0,7	hoher Zusammenhang
0,7 < 1	sehr hoher Zusammenhang

R-squared:
F-statistic prüft, ob das Modell insgesamt einen Erklärungsbeitrag leistet.
t: t-wert: 
p: p-wert:
Die Signifikanz (p-Wert) sollte einen möglichst kleinen Wert (<0,05) haben. Wenn dem so ist, leistet das Regressionsmodell einen Erklärungsbeitrag.
Ist die Signifikanz über 0,05, leistet das Regressionsmodell keinen signifikanten Erklärungsbeitrag und das Verfahren bzw. die weitere Interpretation ist abzubrechen

Interpretation:

Der Koeffizient der unabhängigen Variable (Anzahl der Gruppe) wird wie folgt interpretiert: Wenn (Gruppe) um eine Einheit steigt (ein zusätzlicher Wert hinzukommt), so nimmt die Anzahl des bestimmten Themengebiets zu. 

# Multiple lineare Regression.


Vorraussetzung: Zwischen UA und AV besteht ein linearer Zusammenhang.
Die multiple Regression ist wie die lineare Regression, aber mit mehr als einem UV, was bedeutet, dass wir versuchen, einen Wert basierend auf zwei oder mehr Variablen vorherzusagen.

In [ ]:
import pandas
from sklearn import linear_model


X = df3[['Weiblich', 'Anzahl der Gruppen (Sportler, Musiker usw.)']] #UV
y = df3['Anzahl der Begriffe (Familie)'] #AV

regr = linear_model.LinearRegression()
regr.fit(X, y)


#Vorhersage: Anzahl der Begriffe von der Familie wo das weibliche Geschlecht 50x vorkommt und die Anzahl der Gruppen ist 1300x.
predict = regr.predict([[200, 400]])

print(predict)

Wir haben vorausgesagt, dass das weibliche Geschlecht, welches 200x in den Berufsbezeichnungen vorkommt etwa 2169x den Begriff Familie in den Suggestion enthält.

# Logistische Regression.


Beispiel: Vorhersage des Geschlechts einer Berufsgruppe zwischen 3 verschiedenen Themen.

# Multivariate Regression.

## Berechnung

# LDA mit pyLDavis

In [ ]:
#Quelle: https://nbviewer.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb#topic=0&lambda=1&term=
#Quelle: https://github.com/bmabey/pyLDAvis/issues/4
#!pip install pyLDAvis
from imp import reload
import gensim
from gensim import corpora, models, similarities
import pyLDAvis.gensim as gensimvis
import pyLDAvis
import sys


documents = df['tokens']

stoplist = set('for a of the and to in'.split())
texts = [[word for word in document if word not in stoplist]
         for document in documents]

from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, passes=10)

vis_data = gensimvis.prepare(lda, corpus, dictionary)

In [ ]:
vis_data

# Visulaisierungen

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
  
x = np.arange(5) 
y1 = [34, 56, 12, 89, 21] 
y2 = [12, 56, 78, 45, 22] 
y3 = [14, 23, 45, 25, 24] 
width = 0.2
  
plt.bar(x-0.2, y1, width, color='cyan') 
plt.bar(x, y2, width, color='orange') 
plt.bar(x+0.2, y3, width, color='green') 
plt.xticks(x, ['Sportler', 'Musiker', 'Influencer', 'Politiker', 'usw.']) 
plt.xlabel("Berufsgruppen") 
plt.ylabel("Scores") 
plt.legend(["Familie", "Politik", "Technik"]) 
plt.show()